# Property mutation

Sebastian Esser

TUM CMS; July 2021

## Scenario
Sample notebook that parses two IFC models, calculates the diff and integrates the abstracted Patch into a target model.

# Table of Content
- Init database connection
- parse two model revisions into the graph DB
- abstract difference between both revisions and create patch
- parse initial model once again and apply patch
- create SPF-based representation after patch integration


In [ ]:
# Init databse connection
# prerequisites:
# - neo4j desktop up and running

# all necessary imports
from IfcGraphInterface.Graph2IfcTranslator import Graph2IfcTranslator
from IfcGraphInterface.Ifc2GraphTranslator import IFCGraphGenerator
from PatchManager.PatchGenerator import PatchGenerator
from PatchManager.PatchIntegrator import PatchIntegrator
from neo4jGraphDiff.GraphDiff import GraphDiff
from neo4j_middleware.neo4jConnector import Neo4jConnector

# connect to database
connector = Neo4jConnector()
connector.connect_driver()


# Parse IFC models in DB

In [ ]:
model_name_init = './00_sampleData/IFC_stepP21/GeomRepresentation_05/cube_single.ifc'
model_name_updated = './00_sampleData/IFC_stepP21/GeomRepresentation_05/cube_double.ifc'

graphGenerator_init = IFCGraphGenerator(connector, model_name_init, None)
graphGenerator_init.generateGraph()

graphGenerator_updated = IFCGraphGenerator(connector, model_name_updated, None)
graphGenerator_updated.generateGraph()
print('Graphs generated successfully')


# Calculate Patch



In [ ]:
label_init = 'ts20210623T091748'
label_updated = 'ts20210623T091749'
print_diff_report = True

diff = GraphDiff(label_init=label_init, label_updated=label_updated)
report = diff.run_diff(connector=connector)
if print_diff_report:
    report.print_report()

# 3 -- generate patch --
patch_generator = PatchGenerator(connector=connector)
patch = patch_generator.create_patch_from_graph_diff(report)


# Apply Patch


In [ ]:
graphGenerator_artificial = IFCGraphGenerator(connector, model_name_init, None)
graphGenerator_artificial.label = 'ts9999'
label_toBeUpdated = graphGenerator_artificial.generateGraph()

print(patch.operations[0].pattern.to_cypher_merge(timestamp=label_toBeUpdated))
exit()

integrator = PatchIntegrator(connector=connector)
# integrator.apply_patch(incoming_patch)



# Export graph to SPF


In [ ]:
parser = Graph2IfcTranslator()

# Disconnect from db



In [ ]:
connector.disconnect_driver()
